## Analyse quality score across genome from bam file

In [1]:
import os
import sys

import re
import pysam

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

#load reusable functions
from library_functions import cigar_positions, sam_positional_error, calc_error_probabilities, cigar_positions
# inline stylization
#%matplotlib inline
#plt.rcParams['figure.dpi'] = 150

## Settings

In [7]:
#List location where all BAM files should be found
base_dir = "/home/dan/WGS/analyses/QC"
data_dir = os.path.join(base_dir+"/data")
data_out = os.path.join(base_dir+"/output")
figs_out  = os.path.join(base_dir+"/figs")
#File format
match=re.compile(".primertrimmed.rg.sorted.bam$")

In [8]:
print(data_dir)

/home/dan/WGS/analyses/QC/data


## Extract and plot data

In [14]:
#Match the correct files from the data
for file in [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(data_dir)) for f in fn]:
    if match.search(file):
        #Define all input and output files
        head_tail=os.path.split(file) #Split file and path
        sample_id=head_tail[1] #Get the filename
        sample_id=re.sub(match,'',sample_id) #Remove the match
        sam_file = os.path.join(data_out, sample_id+".sam")
        bam_file = os.path.join(data_dir, file)
        plot_file = os.path.join(figs_out, sample_id+".QC.pdf")
        csv_file = os.path.join(data_out, sample_id+".df.csv")
        
        #Convert BAM to SAM
        cmd = "samtools view %s -o %s" % (bam_file, sam_file)
        os.system(cmd)
        
        #Extract positional error_probs from SAM file
        positions, error_probs = sam_positional_error(sam_file)
        
        #Need to create a table with sample ID and then various outputs (none of these are working)
        
        #Extract number of reads in the bam file
        cmd = "samtools view -c %s" % (bam_file)
        ReadNumber = os.system(cmd)
        
        #Total number of base pairs contained in the bam file
        cmd = "samtools stats %s | grep 'bases mapped (cigar):'" % (bam_file)
        Totalbp = os.system(cmd)
        
        #Delete the SAM file
        os.remove(sam_file)
        
        
        
        ################ NEXT SECTION IS PROBABLY SPURIOUS
        
        if plot==1: 

            #Concatenate all data
            positions = np.concatenate(positions)
            error_probs = np.concatenate(error_probs)
            keep = positions > 0
            positions = positions[keep]
            error_probs = error_probs[keep]

            #Put outputs into a dataframe
            df = (pd.DataFrame({"position": positions, "error_prob": error_probs})
                  .groupby("position")
                  .mean()
                  .reset_index()
                  .sort_values("position")
                 )
            #Add coverage #####HOW TO DO THIS DIRECTLY / I DON'T UNDERSTAND HOW COVERAGE IS CALCULATED
            df["coverage"] = (pd.DataFrame({"position": positions, "error_prob": error_probs})
                              .groupby("position")
                              .size()
                              .values
                             )
            #Output the dataframe
            test_data = df.to_csv(csv_file, index = True) 

            #Plot the data
            fig, ax = plt.subplots(1, 1, figsize=(8, 4))

            ax.plot(df["position"], df["error_prob"])
            ax.set_xlabel("Genomic Position")
            ax.set_ylabel("Error Probability")
            ax.set_title(f"Sample = {sample_id}", loc="left")

            axm = ax.twinx()
            axm.fill_between(x=df["position"],
                             y1=np.repeat(0, df.shape[0]),
                             y2=df["coverage"],
                             color='darkgrey', alpha=0.5)
            axm.set_ylabel("Coverage (grey)")

            #Export plot
            plt.savefig(plot_file)
    

/home/dan/WGS/analyses/QC/data/25267.primertrimmed.rg.sorted.bam
0
0
